# 📚 Documentation Processing Pipeline

This notebook contains all the steps to process documentation: crawl → chunk → embed upload

## Overview
- **Step 1**: Crawl documentation from URLs
- **Step 2**: Chunk documents into smaller pieces
- **Step 3**: Create embeddings and upload to Pinecone

## Prerequisites
Make sure you have:
- `.env` file with your API keys
- Appwrite database and storage set up
- Pinecone index created


In [9]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58

In [4]:
import os
import sys
import json
import time
import logging
from dotenv import load_dotenv
from appwrite_service import appwrite_service

# Load environment variables
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ Setup complete")


✅ Setup complete


In [3]:
# Documentation URL to process
DOCUMENTATION_URL = "https://react.dev/learn"

print(f"📚 Processing: {DOCUMENTATION_URL}")


📚 Processing: https://react.dev/learn


## Step 1: Crawl Documentation

This step crawls the documentation website and extracts all the content.

In [5]:
# Import crawling functions
from crawl_docs import crawl_documentation

print("🕷️  Step 1: Crawling Documentation")
print("=" * 50)
print(f"📚 URL: {DOCUMENTATION_URL}")

# Check if raw docs already exist
if appwrite_service.docs_already_exist(DOCUMENTATION_URL):
    print("✅ Raw documents already exist, skipping crawl...")
    crawl_success = True
else:
    # Start crawling
    start_time = time.time()
    crawl_success = crawl_documentation(DOCUMENTATION_URL)
    crawl_time = time.time() - start_time

    if crawl_success:
        print(f"✅ Crawl completed successfully in {crawl_time:.2f} seconds!")
        print(f"📄 Raw documents saved to storage")
    else:
        print(f"❌ Crawl failed!")

print(f"📊 Crawl step result: {'✅ Success' if crawl_success else '❌ Failed'}")


🕷️  Step 1: Crawling Documentation
📚 URL: https://react.dev/learn
✅ Raw documents already exist, skipping crawl...
📊 Crawl step result: ✅ Success


## Step 2: Chunk Documents

This step takes the raw documents and splits them into smaller chunks for better processing.


In [6]:
# Import chunking functions
from chunk_docs import chunk_and_save_docs

print("🔪 Step 2: Chunking Documents")
print("=" * 50)
print(f"📚 URL: {DOCUMENTATION_URL}")

# Check if chunks already exist
if appwrite_service.chunks_already_exist(DOCUMENTATION_URL):
    print("✅ Chunks already exist, skipping chunking...")
    chunk_success = True
else:
    # Start chunking
    start_time = time.time()
    chunk_success = chunk_and_save_docs(DOCUMENTATION_URL)
    chunk_time = time.time() - start_time

    if chunk_success:
        print(f"✅ Chunking completed successfully in {chunk_time:.2f} seconds!")
        print(f"🔪 Documents chunked and saved to storage")
    else:
        print(f"❌ Chunking failed!")

print(f"📊 Chunk step result: {'✅ Success' if chunk_success else '❌ Failed'}")


🔪 Step 2: Chunking Documents
📚 URL: https://react.dev/learn
✅ Chunks already exist, skipping chunking...
📊 Chunk step result: ✅ Success


## Step 3: Create Embeddings and Upload

This step creates embeddings for the chunks and uploads them to Pinecone for vector search.

In [10]:
# Import embedding functions
from embed_upload import embed_and_upload_chunks

print("🧠 Step 3: Creating Embeddings and Uploading")
print("=" * 50)
print(f"📚 URL: {DOCUMENTATION_URL}")

# Start embedding and uploading
start_time = time.time()
embed_success = embed_and_upload_chunks(DOCUMENTATION_URL)
embed_time = time.time() - start_time

if embed_success:
    print(f"✅ Embedding and upload completed successfully in {embed_time:.2f} seconds!")
    print(f"🧠 Embeddings created and uploaded to Pinecone")
else:
    print(f"❌ Embedding and upload failed!")

print(f"📊 Embed step result: {'✅ Success' if embed_success else '❌ Failed'}")


🧠 Step 3: Creating Embeddings and Uploading
📚 URL: https://react.dev/learn


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

✅ Embedding and upload completed successfully in 585.43 seconds!
🧠 Embeddings created and uploaded to Pinecone
📊 Embed step result: ✅ Success


## Step 4: Save Completion Status

This step saves the completion status to the database so the documentation appears as available in the frontend.

In [11]:
print("💾 Step 4: Saving Completion Status")
print("=" * 50)
print(f"📚 URL: {DOCUMENTATION_URL}")

# Get chunk count for completion status
chunks = appwrite_service.get_all_chunks(DOCUMENTATION_URL)
chunks_count = len(chunks) if chunks else 0
print(f"📊 Found {chunks_count} chunks")

# Save completion status
if embed_success and chunks_count > 0:
    completion_success = appwrite_service.save_completion_status(DOCUMENTATION_URL, chunks_count)

    if completion_success:
        print(f"✅ Completion status saved successfully!")
        print(f"📝 Documentation is now available for chat")
    else:
        print(f"❌ Failed to save completion status")
else:
    print(f"⚠️  Skipping completion status - embedding failed or no chunks found")
    completion_success = False

print(f"📊 Completion step result: {'✅ Success' if completion_success else '❌ Failed'}")


💾 Step 4: Saving Completion Status
📚 URL: https://react.dev/learn
📊 Found 15841 chunks
✅ Completion status saved successfully!
📝 Documentation is now available for chat
📊 Completion step result: ✅ Success


## Final Results

Let's check the overall pipeline results and verify everything worked correctly.


In [12]:
print("🎉 Pipeline Results")
print("=" * 50)

# Check all steps
steps = {
    "Crawl": crawl_success,
    "Chunk": chunk_success,
    "Embed": embed_success,
    "Completion": completion_success
}

all_success = all(steps.values())

print(f"📊 Step Results:")
for step, success in steps.items():
    status = "✅ Success" if success else "❌ Failed"
    print(f"   {step}: {status}")

print(f"\n🎯 Overall Result: {'✅ SUCCESS' if all_success else '❌ FAILED'}")

if all_success:
    print(f"\n🎉 Congratulations! {DOCUMENTATION_URL} is now fully processed and available for chat!")
    print(f"📝 You can now use this documentation in the frontend interface.")
else:
    print(f"\n⚠️  Some steps failed. Please check the logs above and try again.")

# Verify completion status
is_available = appwrite_service.is_fully_processed(DOCUMENTATION_URL)
print(f"\n🔍 Verification: {'✅ Available for chat' if is_available else '❌ Not available'}")


🎉 Pipeline Results
📊 Step Results:
   Crawl: ✅ Success
   Chunk: ✅ Success
   Embed: ✅ Success
   Completion: ✅ Success

🎯 Overall Result: ✅ SUCCESS

🎉 Congratulations! https://react.dev/learn is now fully processed and available for chat!
📝 You can now use this documentation in the frontend interface.

🔍 Verification: ✅ Available for chat


## Utility Functions

Here are some utility functions to check status and manage the pipeline.


In [13]:
def check_documentation_status(url):
    """Check the current status of documentation processing"""
    print(f"\n📊 Status Check for: {url}")
    print("=" * 50)

    # Check raw docs
    raw_exists = appwrite_service.docs_already_exist(url)
    print(f"📄 Raw documents: {'✅ Exists' if raw_exists else '❌ Not found'}")

    # Check chunks
    chunks_exist = appwrite_service.chunks_already_exist(url)
    print(f"🔪 Chunks: {'✅ Exists' if chunks_exist else '❌ Not found'}")

    # Check completion status
    is_processed = appwrite_service.is_fully_processed(url)
    print(f"✅ Completion status: {'✅ Exists' if is_processed else '❌ Not found'}")

    # Overall status
    if is_processed:
        print(f"\n🎉 Documentation is fully processed and ready for questions!")
    elif raw_exists and chunks_exist:
        print(f"\n⚠️  Raw documents and chunks exist but need embedding.")
    elif raw_exists:
        print(f"\n⚠️  Raw documents exist but need chunking and embedding.")
    else:
        print(f"\n❌ Documentation needs to be crawled first.")

# Check status of current documentation
check_documentation_status(DOCUMENTATION_URL)



📊 Status Check for: https://react.dev/learn
📄 Raw documents: ✅ Exists
🔪 Chunks: ✅ Exists
✅ Completion status: ✅ Exists

🎉 Documentation is fully processed and ready for questions!


## Process Different Documentation

You can easily process different documentation by changing the URL and running the pipeline again.

In [ ]:
# Predefined documentation sets
PREDEFINED_DOCS = {
    "React": "https://react.dev/learn",
    "Python": "https://docs.python.org/3/",
    "Node.js": "https://nodejs.org/en/docs/",
    "Vue.js": "https://vuejs.org/guide/",
    "Django": "https://docs.djangoproject.com/en/stable/",
    "FastAPI": "https://fastapi.tiangolo.com/",
    "MongoDB": "https://docs.mongodb.com/",
    "PostgreSQL": "https://www.postgresql.org/docs/",
}

print("📚 Available Documentation Sets:")
for name, url in PREDEFINED_DOCS.items():
    is_available = appwrite_service.is_fully_processed(url)
    status = "✅ Available" if is_available else "❌ Not Available"
    print(f"  {name}: {status}")

print("\n💡 To process a different documentation set:")
print("1. Change DOCUMENTATION_URL to the desired URL")
print("2. Run the pipeline cells again")


## Summary

This notebook provides a complete pipeline for processing documentation:

1. **Crawl**: Extract content from documentation websites
2. **Chunk**: Split content into manageable pieces
3. **Embed**: Create vector embeddings and upload to Pinecone
4. **Complete**: Save completion status for frontend availability

### Usage:
- Change `DOCUMENTATION_URL` to process different documentation
- Run cells in order from top to bottom
- Check results and status using utility functions

### Next Steps:
- Run the frontend app to chat with processed documentation
- Process additional documentation sets as needed
- Monitor and maintain the pipeline
